# Tech Intro

## Main tables

In [1]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [2]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = lead_list[0] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [3]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

22/11/19 13:01:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/19 13:01:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/11/19 13:01:24 WARN Utils: Service 'SparkUI' could not bind on port 4080. Attempting port 4081.
22/11/19 13:01:24 WARN Utils: Service 'SparkUI' could not bind on port 4081. Attempting port 4082.
22/11/19 13:01:24 WARN Utils: Service 'SparkUI' could not bind on port 4082. Attempting port 4083.
22/11/19 13:01:24 WARN Utils: Service 'SparkUI' could not bind on port 4083. Attempting port 4084.
22/11/19 13:01:24 WARN Utils: Service 'SparkUI' could 

## Read the spec-file from Hadoop and show some data sample

In [4]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+--------+--------------------------------+-------+-----------+-------------+-------------------+---------------+----------------+---------+-----+----+-------+-----------------+-------------+------+------+-----+-----+-------+----------+-----------+---------+--------+--------------+-----------------+-----------------+---------+-------+-------+------------+---------------+-------------------+------------+------------+------------------+-------+----------+-----------+
|id      |name                            |ag_name|date_create|last_modified|responsible_user_id|created_user_id|modified_user_id|status_id|price|sort|deleted|leads_pipeline_id|finish_reason|source|status|rg_id|agent|address|closed_at |create_type|filial_id|order_id|agent_accepted|agent_accepted_at|ag_pipe_status_id|ag_status|ag_type|comment|date_meeting|who_see_comment|leads_source_spr_id|created_chat|agent_module|call_center_module|lead_id|company_id|is_exported|
+--------+--------------------------------+-------+---------

## Show columns' properties

In [5]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ag_name: string (nullable = true)
 |-- date_create: string (nullable = true)
 |-- last_modified: string (nullable = true)
 |-- responsible_user_id: string (nullable = true)
 |-- created_user_id: string (nullable = true)
 |-- modified_user_id: string (nullable = true)
 |-- status_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- sort: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- leads_pipeline_id: string (nullable = true)
 |-- finish_reason: string (nullable = true)
 |-- source: string (nullable = true)
 |-- status: string (nullable = true)
 |-- rg_id: string (nullable = true)
 |-- agent: string (nullable = true)
 |-- address: string (nullable = true)
 |-- closed_at: string (nullable = true)
 |-- create_type: string (nullable = true)
 |-- filial_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- agent_accepted: string (nullable

## Total rows (count)

In [6]:
eda_df.count()

981633

## Simple func to execute and show the result of the SQL-instruction

In [7]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [8]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [9]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|981633  |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [10]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,name,ag_name,date_create,last_modified,responsible_user_id,created_user_id,modified_user_id,status_id,price,...,comment,date_meeting,who_see_comment,leads_source_spr_id,created_chat,agent_module,call_center_module,lead_id,company_id,is_exported
0,0,29,587301,2,30,33580,359328,66277,568645,964105,...,718389,596682,0,696131,0,0,0,824129,0,0


In [11]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,name,ag_name,date_create,last_modified,responsible_user_id,created_user_id,modified_user_id,status_id,price,...,comment,date_meeting,who_see_comment,leads_source_spr_id,created_chat,agent_module,call_center_module,lead_id,company_id,is_exported
0,0,1,0,0,0,0,17,13,0,583,...,1451,114,0,0,977899,429457,552176,0,0,981132


In [12]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,name,ag_name,date_create,last_modified,responsible_user_id,created_user_id,modified_user_id,status_id,price,...,comment,date_meeting,who_see_comment,leads_source_spr_id,created_chat,agent_module,call_center_module,lead_id,company_id,is_exported
0,0,948,0,0,0,0,0,0,0,0,...,206064,0,0,0,0,0,0,0,0,0


In [13]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
name,29,1,948
ag_name,587301,0,0
date_create,2,0,0
last_modified,30,0,0
responsible_user_id,33580,0,0
created_user_id,359328,17,0
modified_user_id,66277,13,0
status_id,568645,0,0
price,964105,583,0


In [16]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [17]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
name,0.003,0.000,0.097
ag_name,59.829,0.000,0.000
date_create,0.000,0.000,0.000
last_modified,0.003,0.000,0.000
responsible_user_id,3.421,0.000,0.000
created_user_id,36.605,0.002,0.000
modified_user_id,6.752,0.001,0.000
status_id,57.928,0.000,0.000
price,98.214,0.059,0.000


### SQL-instructions

In [18]:
#Find dupes or misunderstanding or useless static values 981 633 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|981633            |
+------------------+



+--------------------+
|count(DISTINCT name)|
+--------------------+
|587914              |
+--------------------+



+-----------------------+
|count(DISTINCT ag_name)|
+-----------------------+
|341225                 |
+-----------------------+



+---------------------------+
|count(DISTINCT date_create)|
+---------------------------+
|943987                     |
+---------------------------+



+-----------------------------+
|count(DISTINCT last_modified)|
+-----------------------------+
|895297                       |
+-----------------------------+

+-----------------------------------+
|count(DISTINCT responsible_user_id)|
+-----------------------------------+
|11586                              |
+-----------------------------------+

+-------------------------------+
|count(DISTINCT created_user_id)|
+-------------------------------+
|11696                          |
+-------------------------------+

+--------------------------------+
|count(DISTINCT modified_user_id)|
+--------------------------------+
|8200                            |
+--------------------------------+

+-------------------------+
|count(DISTINCT status_id)|
+-------------------------+
|70                       |
+-------------------------+

+---------------------+
|count(DISTINCT price)|
+---------------------+
|2568                 |
+---------------------+

+--------------------+
|count(DISTINCT 

+-------------------------+
|count(DISTINCT closed_at)|
+-------------------------+
|555670                   |
+-------------------------+

+---------------------------+
|count(DISTINCT create_type)|
+---------------------------+
|8                          |
+---------------------------+

+-------------------------+
|count(DISTINCT filial_id)|
+-------------------------+
|209                      |
+-------------------------+

+------------------------+
|count(DISTINCT order_id)|
+------------------------+
|2                       |
+------------------------+

+------------------------------+
|count(DISTINCT agent_accepted)|
+------------------------------+
|2                             |
+------------------------------+

+---------------------------------+
|count(DISTINCT agent_accepted_at)|
+---------------------------------+
|224140                           |
+---------------------------------+

+---------------------------------+
|count(DISTINCT ag_pipe_status_id)|
+-----------

In [100]:
#Find dupes or misunderstanding or useless static values 981 633 total

execute("SELECT DISTINCT(date_meeting)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY date_meeting ASC")

execute("SELECT DISTINCT(ag_type)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY ag_type DESC")

execute("SELECT DISTINCT(status)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY status ASC")

execute("SELECT price\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY price DESC")

execute("SELECT last_modified\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY last_modified ASC")

execute("SELECT date_create\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY date_create ASC")

execute("SELECT ag_name\
        FROM eda_sql\
        WHERE ag_name like '8%'\
        ORDER BY ag_name ASC")

execute("SELECT name\
        FROM eda_sql\
        WHERE name like '1%'\
        ORDER BY name ASC")

execute("SELECT name\
        FROM eda_sql\
        WHERE name != ''\
        ORDER BY name ASC")

execute("SELECT lead_id,\
        count(lead_id) AS quantity \
        FROM eda_sql\
        GROUP BY lead_id\
        HAVING count(lead_id) > 1 \
        ORDER BY quantity DESC")


+------------+
|date_meeting|
+------------+
|null        |
|0           |
|1539118800  |
|1578942000  |
|1586725200  |
|1593370800  |
|1595451600  |
|1595962800  |
|1596142800  |
|1598900400  |
|1599418800  |
|1602450000  |
|1602709200  |
|1602788400  |
|1603306800  |
|1603746000  |
|1604948400  |
|1605643200  |
|1605733200  |
|1606942800  |
+------------+
only showing top 20 rows

+-------+
|ag_type|
+-------+
|4      |
|3      |
|2      |
|1      |
|0      |
+-------+

+------+
|status|
+------+
|0     |
|1     |
|2     |
|3     |
|32767 |
|4     |
|5     |
|6     |
|7     |
+------+

+---------+
|price    |
+---------+
|9999000  |
|9990000  |
|9990000  |
|9990000  |
|9990000  |
|999000   |
|999000   |
|99900    |
|99900    |
|998000000|
|998000   |
|998000   |
|998000   |
|99522    |
|9950000  |
|99500    |
|99489    |
|99200    |
|99160    |
|9900000  |
+---------+
only showing top 20 rows

+-------------+
|last_modified|
+-------------+
|null         |
|null         |
|null      

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [142]:
spark.stop()

I1110 07:32:53.662405 29725 sched.cpp:2174] Asked to stop the driver
I1110 07:32:53.662559 30188 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-56916
